In [42]:
import requests
import os
from bs4 import BeautifulSoup
import hashlib
import pandas as pd

In [19]:



pathSM = "data/sitemap.xml"
if os.path.isfile(pathSM):
    print(pathSM, ": already done")
else:
    url = 'https://www.mottmac.com/sitemap.xml'
    r = requests.get(url, allow_redirects=True)
    open(pathSM, 'wb').write(r.content)

pathRobot = "data/robots.txt"
if os.path.isfile(pathRobot):
    print(pathRobot, ": already done")
else:
    url = 'https://www.mottmac.com/robots.txt'
    r = requests.get(url, allow_redirects=True)
    open(pathRobot, 'wb').write(r.content)

data/sitemap.xml : already done
data/robots.txt : already done


In [20]:

with open(pathSM, 'r') as f:
    data = f.read()
pages = BeautifulSoup(data, "xml")

In [39]:
urlList = []
URLs = pages.find_all('url')
for url in URLs:
    ref = url.find_all('xhtml:link', {'rel':'alternate'})[0]
    urlList.append(ref.get('href'))
urlList = list(set(urlList)) 
urlList = [x for x in urlList if not "/job" in x ]
urlList

['https://www.mottmac.com/releases/mott-macdonald-appointed-digital-and-innovation-healthcare-advisor-by-the-leeds-teaching-hospitals-nhs-trust-ltht',
 'https://www.mottmac.com/releases/mott-macdonald-appoints-bruno-trouille-as-senior-hydropower-advisor',
 'https://www.mottmac.com/releases/speyside-biomass-power-station-receives-74-million-funding-uk',
 'https://www.mottmac.com/careers/giving-up-life-in-the-fast-lane',
 'https://www.mottmac.com/en-US/article/37197/bioengineered-oyster-reef-demonstration',
 'https://www.mottmac.com/careers/corporate-services',
 'https://www.mottmac.com/article/466/balfour-beatty-mott-macdonald-revolutionary-s',
 'https://www.mottmac.com/en-US/article/37399/raritan-river-force-main',
 'https://www.mottmac.com/releases/mott-macdonald-successfully-provides-environmental-and-social-advice-on-the-largest-solar-pv-complex-in-africa',
 'https://www.mottmac.com/careers/mott-macdonald-listed-on-glassdoor-best-places-to-work-2023',
 'https://www.mottmac.com/en-US

In [71]:
df = dict()
for url in urlList:
    df[url] = hashlib.md5(url.encode()).hexdigest()
df = pd.DataFrame.from_dict(df, orient='index').reset_index()
df.columns = ["url","hash"]
df.to_parquet("data/pages.parquet.gzip")
df

,url,hash
0,https://www.mottmac.com/releases/mott-macdonal...,ae00a6f326032789c8f62725015567f9
1,https://www.mottmac.com/releases/mott-macdonal...,d5cadd4b35b047fc62d9bac02fb5e522
2,https://www.mottmac.com/releases/speyside-biom...,8fac9a3e4b3dbdc711bb9cac3b1f21ef
3,https://www.mottmac.com/careers/giving-up-life...,29ebaa0e213b404db68baa57a7610624
4,https://www.mottmac.com/en-US/article/37197/bi...,28ba2edb64e266ce881c1c6e039bf505
...,...,...
4002,https://www.mottmac.com/article/11603/projects...,47e3aeb40050521095a7b3ef774520fb
4003,https://www.mottmac.com/releases/national-coll...,b23b4df9e9fedfe5e5dcb33458a99d7a
4004,https://www.mottmac.com/article/642/mott-macdo...,4755783c81de654818b781a126b59006
4005,https://www.mottmac.com/article/529/mott-macdo...,a879e1ea323939740256a188945aec41


In [61]:
for ix,row in df.iterrows():
    name = row["hash"]
    url = row["url"]

    if not os.path.exists("data/cache/"+name):
        page_sourced = requests.get(url).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        with open("data/cache/"+name, 'w') as f:
            f.write(str(html_content))
        print(name,"saved")

b973fe51590acbab718ea9ae695e0e28 saved
67b9b747d9bea46afa1d557753f4f04b saved
3f7010b183bba25690649927edd494f7 saved
4bea9ae08c824d3220653854581515ae saved
02d4b24de5e55b1e7ca02e1a3515a516 saved
596c2f2d889042edc28d73285e7b3e23 saved
61af9f0ffbdbd72a03595187f50acc56 saved
2d474aef61f19d8083dfc4fad0c34682 saved
e2272164fc93da103e3bfe18b2cf8e72 saved
71efb7f5f2a7e9b9a0666c196f797d36 saved
9670e3b3761e5cd5f711daa365796d05 saved
fbe4b3bf534de511b06909e00e48499d saved
908f232300f92eb7cbb3e69269e23de6 saved
da90b74e210de235ca3d5c7e8705afe9 saved
1c2803e3fae41065a3dba694adc63c86 saved
42cc6a8c4586a61c22a05e4c846d0e38 saved
cab6c5a4da66b800dbbbaaa0e504ea05 saved
d2fb4a4ddeacfd2f7194652f3f9944a3 saved
5b92ec30204dad803c8b5af7a5c2e27f saved
5e2ea179cf4a61aa4bbe2a8e3cf672f5 saved
b27f269388b15cca4d35cdd7bb7d5c0c saved
57d105b9dbc3fa6f259f38fc8aabbbec saved
fa8b20ecac32aa064db8729a45f485be saved
58f402bde0b64a6555cc0f6c0a9f790a saved
6efc3223073d78a72073ce87ea3dce59 saved
074a6062580615083820c53f8

KeyboardInterrupt: 

# Cleaning the text

In [66]:
import os

import numpy as np
import pandas as pd
import io
from lxml import html
from bs4 import BeautifulSoup
import trafilatura

In [67]:
files = os.listdir('data/cache/')
file_names = []
for name in files:
    if not ('type' in name):
        file_names.append(name)

In [68]:
articles = []
errors = []
for file_name in file_names:
    with io.open(f'data/cache/{file_name}', mode="r", encoding="utf-8") as f:
        try:
            mytree = html.fromstring("".join(f.readlines()))
        except Exception as e:
            print(e)
            errors.append(file_name)
            continue
        try:
            content = trafilatura.extract(mytree)
            articles.append((file_name, content))
        except Exception as e:
            print(e)
            errors.append(file_name)

In [70]:
dfContent = pd.DataFrame(articles, columns = ['file_name', 'content'])
dfContent.to_parquet("data/content.parquet.gzip")
dfContent

,file_name,content
0,6efc3223073d78a72073ce87ea3dce59,The Port Authority of New York and New Jersey ...
1,ae00a6f326032789c8f62725015567f9,Mott MacDonald has been appointed digital and ...
2,28ba2edb64e266ce881c1c6e039bf505,Opportunity\n“Weak” soils with low bearing cap...
3,1e6f7fb8d576dfad233c9ba771b4367f,"Patrick Natale, Vice President–Business Strate..."
4,ed8ba63d1b8667c4b1b3426b33650ef4,"Merz and McLellan, the South Africa and Botswa..."
5,0631b4860fa150ff4ca0dcb63ad92b4d,A new year at Mott MacDonald is off to a flyin...
6,64931626c55fe748120831b24837f5bf,The next phase of work on a major project to p...
7,5b92ec30204dad803c8b5af7a5c2e27f,Development of the Olse revitalisation program...
8,bbfbf9fdb786c80492a9093dc41c1db5,Mott MacDonald delivered the environmental and...
9,25c4ec8f76594126541c1d6450784af4,"Mott MacDonald programme controls director, Pa..."
